# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
from src.match_names import name_fbref_to_fpl, neutralize_name

# Getting fbref

In [3]:
fbref = pd.read_csv("data/logs_all.csv")

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
fbref.head()

,Unnamed: 0,level_0,index,Date,Day,Comp,Round,Venue,Squad,Opponent,...,OG,PKwon,PKcon,WDL,GoalsTeams,Team Score,Opp Score,CS,Was Home,GW
0,0,0,0.0,2021-08-21,Sat,Premier League,Matchweek 2,Home,Brighton,Watford,...,NaN,NaN,NaN,W,2–0,2,0,1.0,1.0,2
1,1,1,2.0,2021-08-28,Sat,Premier League,Matchweek 3,Home,Brighton,Everton,...,NaN,NaN,NaN,L,0–2,0,2,0.0,1.0,3
2,2,2,5.0,2021-09-19,Sun,Premier League,Matchweek 5,Home,Brighton,Leicester City,...,NaN,NaN,NaN,W,2–1,2,1,0.0,1.0,5
3,3,3,7.0,2021-09-27,Mon,Premier League,Matchweek 6,Away,Brighton,Crystal Palace,...,NaN,NaN,NaN,D,1–1,1,1,0.0,0.0,6
4,4,4,8.0,2021-10-02,Sat,Premier League,Matchweek 7,Home,Brighton,Arsenal,...,NaN,NaN,NaN,D,0–0,0,0,1.0,1.0,7


In [5]:
fbref.shape

(19118, 56)

# Getting FPL

In [6]:
fpl = pd.read_csv("data/fpl_players.csv")

In [7]:
# fpl[(fpl["name"]=="Brenden Aaronson") & (fpl["season"]=="2022-23")]

In [8]:
# removing rows with GW == NaN
fpl = fpl[fpl['GW'].notna()]

In [9]:
# fpl[(fpl["season"] == "2022-23") & (fpl["name"] == "Ben Davies") & (fpl["GW"] == 2)]

In [28]:
fbref["Season"].unique()

array(['2021-22', '2022-23'], dtype=object)

In [10]:
fpl.head()

,Unnamed: 0,Unnamed: 0.1,season,name,position,team,assists,bonus,bps,clean_sheets,...,value,was_home,yellow_cards,GW,xP,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts
0,0,51676.0,2020-21,Aaron Connolly,FWD,Brighton,0,0,-3,0,...,55,True,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,1,51677.0,2020-21,Aaron Cresswell,DEF,West Ham,0,0,11,0,...,50,True,0,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2,51678.0,2020-21,Aaron Mooy,MID,Brighton,0,0,0,0,...,50,True,0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,3,51679.0,2020-21,Aaron Ramsdale,GK,Sheffield Utd,0,0,12,0,...,50,True,0,1,NaN,NaN,NaN,NaN,NaN,NaN
4,4,51680.0,2020-21,Abdoulaye Doucouré,MID,Everton,0,0,20,1,...,55,False,0,1,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
fpl.shape

(76975, 45)

In [12]:
# sorted(fpl["team"].unique())

In [13]:
# sorted(fbref["Squad"].unique())

# Merging

In [14]:
def team_fbref_to_fpl(team_name):
    if team_name == "Nott'ham Forest":
        return "Nott'm Forest"
    if team_name == "Leicester City":
        return "Leicester"
    if team_name == "Leeds United":
        return "Leeds"
    if team_name == "Manchester City":
        return "Man City"
    if team_name == "Manchester Utd":
        return "Man Utd"
    if team_name == "Tottenham":
        return "Spurs"
    if team_name == "Newcastle Utd":
        return "Newcastle"
    
    return team_name

In [15]:
def neutralize_name(name):
    return name.replace(" ", "-").replace("ï", "i").replace("é", "e").replace("á", "a").replace("ó", "o").replace("Á", "A").replace("ø", "o").replace("ć", "c").replace("í", "i").replace("ú", "u").replace("Ć", "C").replace("ã", "a").replace("ğ", "g").replace("ş", "s").replace("É", "E").replace("ñ", "n").replace("Ł", "L").replace("ń", "n").replace("ß", "ss").replace("ç", "c").replace("İ", "I").replace("č", "c").replace("ö", "o").replace("ë", "e").replace("š", "s").replace("ä", "a").replace("Ç", "C").replace("ü", "u").replace("Ø", "O").replace("ú", "u").replace("ú", "u").replace("î", "i")

In [16]:
# add neutral name columns
fpl["neutral_name"] = fpl["name"].apply(neutralize_name)
fbref["neutral_name"] = fbref["Name"].apply(neutralize_name)
fbref["FPL_name"] = fbref["neutral_name"].apply(name_fbref_to_fpl)
fbref["FPL_team"] = fbref["Squad"].apply(team_fbref_to_fpl)
fbref["FPL_opp_team"] = fbref["Opponent"].apply(team_fbref_to_fpl)

In [17]:
# fpl.info()

In [18]:
# fbref.info()

In [19]:
join_columns_fbref = ['FPL_name', 'Season', 'FPL_team', 'Was Home', 'FPL_opp_team']
join_columns_fpl = ['neutral_name', 'season', 'team', 'was_home', 'opp_team_name']

In [20]:
result = fbref.merge(fpl, left_on=join_columns_fbref, right_on=join_columns_fpl, how='left').drop('Unnamed: 0_x', axis = 1)

In [21]:
result

,level_0,index,Date,Day,Comp,Round,Venue,Squad,Opponent,Start,...,was_home,yellow_cards,GW_y,xP,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts,neutral_name_y
0,0,0.0,2021-08-21,Sat,Premier League,Matchweek 2,Home,Brighton,Watford,N,...,True,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Aaron-Connolly
1,1,2.0,2021-08-28,Sat,Premier League,Matchweek 3,Home,Brighton,Everton,N,...,True,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Aaron-Connolly
2,2,5.0,2021-09-19,Sun,Premier League,Matchweek 5,Home,Brighton,Leicester City,N,...,True,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,Aaron-Connolly
3,3,7.0,2021-09-27,Mon,Premier League,Matchweek 6,Away,Brighton,Crystal Palace,N,...,False,0.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,Aaron-Connolly
4,4,8.0,2021-10-02,Sat,Premier League,Matchweek 7,Home,Brighton,Arsenal,N,...,True,0.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,Aaron-Connolly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19113,30,NaN,2023-02-05,Sun,Premier League,Matchweek 22,Home,Tottenham,Manchester City,N,...,True,0.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,Yves-Bissouma
19114,32,NaN,2023-05-06,Sat,Premier League,Matchweek 35,Home,Tottenham,Crystal Palace,N,...,True,0.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,Yves-Bissouma
19115,33,NaN,2023-05-13,Sat,Premier League,Matchweek 36,Away,Tottenham,Aston Villa,N,...,False,0.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,Yves-Bissouma
19116,34,NaN,2023-05-20,Sat,Premier League,Matchweek 37,Home,Tottenham,Brentford,Y,...,True,1.0,37.0,NaN,NaN,NaN,NaN,NaN,NaN,Yves-Bissouma


In [22]:
# result.columns.to_list()

In [25]:
result.shape

(19118, 105)

# Saving merged df to file

In [23]:
# fbref[["Name", "GW", "Season", "Squad", "Total Points"]].to_csv("data/fpl_fbref_players.csv")
result.to_csv("data/fpl_fbref_players.csv")

In [29]:
result["Name"].unique().size

435